# deep learning -Titanic을 해보자!

In [2]:
import pandas as pd
import tensorflow as tf
import warnings
warnings.filterwarnings(action="ignore")  # warning 출력 X

# 데이터 로딩하기 (train data set loading)
test_df = pd.read_csv("./data/titanic/test.csv")
train_df = pd.read_csv("./data/titanic/train.csv")

In [3]:
### 전처리 ### 
train_df["Embarked"].fillna("S",inplace=True)

train_df["Title"]=train_df["Name"].str.extract("([A-Za-z]+)\.")  # +: 1개 이상을 의미  # \.: 점을 의미함.  
  # -> 영문자 나오고 끝에 점찍히는 형태 구해라

# Titanic안에 Mr,Miss,Mrs,other을 각각 0,1,2,3으로 변환
title_mapping_dict = {"Mr":0, "Miss":1,"Mrs":2,"Master":3,"Dr":3,"Rev":3,"Col":3,"Major":3,"Mlle":3,
                      "Don":3,"Jonkheer":3,"Countess":3,"Lady":3,"Mme":3,"Ms":3,"Sir":3,"Capt":3}
train_df["Title"]=train_df["Title"].map(title_mapping_dict)
train_df.drop("Name",axis=1,inplace=True)  # drop: 행, 열 지울 수 있음 
train_df.drop("Ticket",axis=1,inplace=True)
train_df.drop("Cabin",axis=1,inplace=True)
train_df.drop("PassengerId",axis=1,inplace=True)

## 성별 column에 대해 male=0, female=1로 변환
sex_mapping_dict = {"male":0,"female":1}
train_df["Sex"]=train_df["Sex"].map(sex_mapping_dict)
train_df

## Embarked (탑승지역) 컬럼에 대해 S = 0, Q =1, C =2로 변환
# -> 그냥 하면 NaN 2개 있기 때문에 결과 실수형으로 나옴.
embarked_mapping_dict = {"S":0,"Q":1,"C":2}
train_df["Embarked"]=train_df["Embarked"].map(embarked_mapping_dict)
train_df

## Age에는 결측치가 많음! 
age_mean = train_df.groupby("Title")["Age"].mean()
a = train_df[train_df["Title"]==0]["Age"].fillna(age_mean[0])
b = train_df[train_df["Title"]==1]["Age"].fillna(age_mean[1])
c = train_df[train_df["Title"]==2]["Age"].fillna(age_mean[2])
d = train_df[train_df["Title"]==3]["Age"].fillna(age_mean[3])
result_series = pd.concat([a,b,c,d])
train_df["Age"]=result_series.sort_index()


## Age에 대해서 binning 처리 (구간으로 나눠서 숫자 mapping시켜서 숫자 이용하기)
# 고려해야 할 사항 -> 간격 어떻게 설정할거냐?

# Age
# 0~20 : 0
# 20초과~40살 이하 : 1
# 40살 초과~60살 이하 : 2
# 60살 초과 : 3
train_df.loc[train_df["Age"]<=20,"Age"]=0
train_df.loc[(train_df["Age"]>20)&(train_df["Age"]<=40),"Age"]=1
train_df.loc[(train_df["Age"]>40)&(train_df["Age"]<=60),"Age"]=2
train_df.loc[train_df["Age"]>60,"Age"]=3

# # Fare도 binning 처리 해보자
train_df.loc[train_df["Fare"]<=14,"Fare"]=0
train_df.loc[(train_df["Fare"]>14)&(train_df["Fare"]<=65),"Fare"]=1
train_df.loc[(train_df["Fare"]>65),"Fare"]=2


In [6]:
# train,test data set
x_data = train_df.drop("Survived",axis=1,inplace=False).values  # 컬럼삭제, 원본유지 
y_data = train_df["Survived"].values.reshape([-1,1])  # 1차원의 array로 나오므로 2차원의 numpy array로 형태 바꿔줘야 함

# 80 20나누자
train_num = int(train_df.shape[0]*0.8)  # 712개
# test_num = train_df.shape[0]-train_num

# train,test data set
train_x_data = train_df.drop("Survived",axis=1,inplace=False)[:train_num].values  # 컬럼삭제, 원본유지 
test_x_data = train_df.drop("Survived",axis=1,inplace=False)[train_num:].values  

train_y_data = train_df["Survived"][:train_num].values.reshape([-1,1])  # 1차원의 array로 나오므로 2차원의 numpy array로 형태 바꿔줘야 함
test_y_data = train_df["Survived"][train_num:].values.reshape([-1,1])


In [42]:
train_df.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    2
Title       0
dtype: int64

In [10]:
# 그래프를 초기화해보자! 
tf.reset_default_graph()  #  이전에는 w를 random하게 주어서 오류 안떴었는데 이제 초기값 지정해주기때문에 다시 실행하면 오류뜸

# placeholder
X = tf.placeholder(shape=[None,8], dtype=tf.float32)  
Y = tf.placeholder(shape=[None,1], dtype=tf.float32)  

# W & b (Deep & wide)  - depth가 길면 길수록 정확한 값 안나오고 시간이 너무 오래 걸리므로 적절한 크기로 해줘야 한다. 

W1 = tf.get_variable("weight1", 
                     shape=[8,256], 
                     initializer = tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([256]),name="bias1")
layer1 = tf.nn.relu(tf.matmul(X,W1)+b1)

W2 = tf.get_variable("weight2", 
                     shape=[256,256], 
                     initializer = tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]),name="bias2")
layer2 = tf.nn.relu(tf.matmul(layer1,W2)+b2)

W3 = tf.get_variable("weight3", 
                     shape=[256,1], 
                     initializer = tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([1]),name="bias3")
logit = tf.matmul(layer1,W3)+b3
H = tf.nn.relu(logit)  # 사실 sigmoid 쓰나 softmax쓰나 같은 결과. softmax는 전체 확률로 바뀌고 sigmoid에서 가장 큰 값이 softmax에서도 가장 큰 값인건 바뀌지 않음

# cost
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit,labels=Y))  # logistic 이므로 softmax아니고 sigmoid

# train
#train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)  # 이거도 자주쓰임. 크게 성능의 향상 보기엔 어렵지만 좀더 성능 좋다는게 일반적인 생각

# session, 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습
num_of_epoch = 30  # 반복횟수

batch_size = 100  # 데이터 작아서 batch의 의미 없음
for step in range(num_of_epoch): 
    num_of_iter = int(train_num/batch_size)
    cost_val = 0
    
    for i in range(num_of_iter):
        batch_x = train_x_data[batch_size*i:batch_size*(i+1)]  # x,y 100개씩 떼어오기
        batch_y = train_y_data[batch_size*i:batch_size*(i+1)]
        _, cost_val = sess.run([train,cost],feed_dict = {X:batch_x,
                                                         Y:batch_y})
    if step%3==0:
        print("cost:{}".format(cost_val))

          
# for step in range(num_of_epoch):   
#     _, cost_val = sess.run([train,cost],feed_dict = {X:train_x_data,
#                                                      Y:train_y_data})
#     if step%3000==0:
#         print("cost:{}".format(cost_val))

cost:0.5482234358787537
cost:0.4603429436683655
cost:0.41003790497779846
cost:0.40239569544792175
cost:0.4042731523513794
cost:0.40723899006843567
cost:0.41411906480789185
cost:0.40675461292266846
cost:0.4027542769908905
cost:0.4243813455104828


In [8]:
predict = tf.cast(H > 0.5,dtype=tf.float32)
# H가 예측값인데, 0.2가 나오면 1에 가까울 확률이 20%인 것, 0.8이면 1에 가까울 확률이 80%
# 몇을 넘으면 1로 볼것인지는 마음대로 잡아도 되지만, 일반적으로 0.5 많이 한다.
# H>0.5 하면 결과 True, False로 떨어지므로 우리가 가지고 있는 데이터 0,1로 표현해 줘야 한다.
# 실제 데이터와 비교하기 위해 값 바꿔주는 과정 
correct = tf.equal(predict, Y)
accuracy = tf.reduce_mean( tf.cast(correct,dtype=tf.float32) )
# correct도 True, False로 떨어짐 -> correct를 실수값으로 다시 바꿔주는 과정

print("정확도:{}".format(sess.run(accuracy,feed_dict={X:test_x_data,
                                                  Y:test_y_data})))


정확도:0.8379888534545898


In [114]:
test_df["Embarked"].fillna("S",inplace=True)

test_df["Title"]=test_df["Name"].str.extract("([A-Za-z]+)\.")
# Titanic안에 Mr,Miss,Mrs,other을 각각 0,1,2,3으로 변환
title_mapping_dict = {"Mr":0, "Miss":1,"Mrs":2,"Master":3,"Dr":3,"Rev":3,"Col":3,"Major":3,"Mlle":3,
                      "Don":3,"Jonkheer":3,"Countess":3,"Lady":3,"Mme":3,"Ms":3,"Sir":3,"Capt":3}
test_df["Title"]=test_df["Title"].map(title_mapping_dict)
test_df["Title"].fillna(3,inplace=True)
test_df.drop("Name",axis=1,inplace=True)  # drop: 행, 열 지울 수 있음 
test_df.drop("Ticket",axis=1,inplace=True)
test_df.drop("Cabin",axis=1,inplace=True)
test_df.drop("PassengerId",axis=1,inplace=True)

## 성별 column에 대해 male=0, female=1로 변환
sex_mapping_dict = {"male":0,"female":1}
test_df["Sex"]=test_df["Sex"].map(sex_mapping_dict)

## Embarked (탑승지역) 컬럼에 대해 S = 0, Q =1, C =2로 변환
# -> 그냥 하면 NaN 2개 있기 때문에 결과 실수형으로 나옴.
embarked_mapping_dict = {"S":0,"Q":1,"C":2}
test_df["Embarked"]=test_df["Embarked"].map(embarked_mapping_dict)

## 결측치 채우기 - age, fare
fare_mean = train_df.groupby("Pclass")["Fare"].mean()
#plt.boxplot(test_df["Fare"].dropna(how="any",inplace=False))
#test_df["Fare"].fillna(fare_mean)

fare_mean = test_df.groupby("Pclass")["Fare"].mean()
a = test_df[test_df["Pclass"]==1]["Fare"].fillna(fare_mean[1])
b = test_df[test_df["Pclass"]==2]["Fare"].fillna(fare_mean[2])
c = test_df[test_df["Pclass"]==3]["Fare"].fillna(fare_mean[3])
result_series = pd.concat([a,b,c])
test_df["Fare"]=result_series.sort_index()

age_mean = test_df.groupby("Title")["Age"].mean()
a = test_df[test_df["Title"]==0]["Age"].fillna(age_mean[0])
b = test_df[test_df["Title"]==1]["Age"].fillna(age_mean[1])
c = test_df[test_df["Title"]==2]["Age"].fillna(age_mean[2])
d = test_df[test_df["Title"]==3]["Age"].fillna(age_mean[3])
result_series = pd.concat([a,b,c,d])
test_df["Age"]=result_series.sort_index()


## Age에 대해서 binning 처리 (구간으로 나눠서 숫자 mapping시켜서 숫자 이용하기)
# 고려해야 할 사항 -> 간격 어떻게 설정할거냐?
test_df.loc[test_df["Age"]<=20,"Age"]=0
test_df.loc[(test_df["Age"]>20)&(test_df["Age"]<=40),"Age"]=1
test_df.loc[(test_df["Age"]>40)&(test_df["Age"]<=60),"Age"]=2
test_df.loc[test_df["Age"]>60,"Age"]=3

# Fare도 binning 처리 해보자
# test_df.loc[train_df["Fare"]<=100,"Fare"]=0
# test_df.loc[(test_df["Fare"]>100)&(test_df["Fare"]<=200),"Fare"]=1
# test_df.loc[(test_df["Fare"]>200)&(test_df["Fare"]<=300),"Fare"]=2
# test_df.loc[(test_df["Fare"]>300),"Fare"]=3

# # Fare도 binning 처리 해보자
test_df.loc[test_df["Fare"]<=14,"Fare"]=0
test_df.loc[(test_df["Fare"]>14)&(test_df["Fare"]<=65),"Fare"]=1
test_df.loc[(test_df["Fare"]>65),"Fare"]=2

In [115]:
x_data = test_df.values  # 컬럼삭제, 원본유지 

result_H = sess.run(H, feed_dict={X:x_data})      

predict = tf.cast(result_H > 0.5,dtype=tf.int32)
result = sess.run(predict)


In [117]:
test_id = pd.read_csv("./data/titanic/test.csv")["PassengerId"]

# concat(): 연결 -> 연결할 것을 list 형태로
s1 = pd.DataFrame(test_id)
s2 = pd.DataFrame(result)
result = pd.concat([s1,s2],axis=1)
#result = result.rename({'Survived':'0'},axis='columns')
result.columns=['PassengerId','Survived']
result


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [118]:
result.to_csv('./data/titanic/submission_deep.csv',
                 sep=',',
                 na_rep='NaN', 
                 columns = ['PassengerId','Survived'], # columns to write
                 index = False) # do not write index



# dropout 해보자

In [145]:
x_data = train_df.drop("Survived",axis=1,inplace=False).values  # 컬럼삭제, 원본유지 
y_data = train_df["Survived"].values.reshape([-1,1])  # 1차원의 array로 나오므로 2차원의 numpy array로 형태 바꿔줘야 함


In [146]:
# 그래프를 초기화해보자! 
tf.reset_default_graph() 

# placeholder
X = tf.placeholder(shape=[None,8], dtype=tf.float32)  # 28*28 = 784
Y = tf.placeholder(shape=[None,1], dtype=tf.float32)  # 0-9
keep_rate = tf.placeholder(dtype=tf.float32) # 값 1개 scalar값이므로 shape 안줘도 된다. -> drop out 비율
    # cpu_env에서는 tensorflow 버전 gpu랑 달라서 dout_rate 말고 keep_rate 쓰자. keep_prob로 써야 함  

# W & b (Deep & wide) 
W1 = tf.get_variable("weight1", 
                     shape=[8,100], 
                     initializer = tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([100]),name="bias1")
_layer1 = tf.nn.relu(tf.matmul(X,W1)+b1)
layer1 = tf.nn.dropout(_layer1,keep_prob=keep_rate) # 256개  output을 다 뽑아내지 않겠다. node를 아예 삭제하는게 아니라 기능을 상실시키는 것
                                         # rate=0 하면 다 살아있는거   0.3하면 30% 죽이는거 -> 다음으로 30프로 죽여서 넘기겠다.

# W2 = tf.get_variable("weight2", 
#                      shape=[256,256], 
#                      initializer = tf.contrib.layers.xavier_initializer())
# b2 = tf.Variable(tf.random_normal([256]),name="bias2")
# _layer2 = tf.nn.relu(tf.matmul(layer1,W2)+b2)
# layer2 = tf.nn.dropout(_layer2,keep_prob=keep_rate) 

W3 = tf.get_variable("weight3", 
                     shape=[100,1], 
                     initializer = tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([1]),name="bias3")  # 맨 마지막에는 dropout해주지 않는다. -> 다 넘겨야 하므로
logit = tf.matmul(layer1,W3)+b3
H = tf.nn.relu(logit)  

# cost
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit,labels=Y))  

# train
#train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
train = tf.train.AdamOptimizer(learning_rate=0.05).minimize(cost)  # 이거도 자주쓰임. 크게 성능의 향상 보기엔 어렵지만 좀더 성능 좋다는게 일반적인 생각

# session, 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습
num_of_epoch = 5000  # 반복횟수
# batch_size = 100

# for step in range(num_of_epoch): 
#     num_of_iter = int(train_num/batch_size)
#     cost_val = 0
    
#     for i in range(num_of_iter):
#         batch_x = train_x_data[batch_size*i:batch_size*(i+1)]  # x,y 100개씩 떼어오기
#         batch_y = train_y_data[batch_size*i:batch_size*(i+1)]
#         _, cost_val = sess.run([train,cost],feed_dict = {X:batch_x,
#                                                          Y:batch_y,
#                                                          keep_rate:0.6}) # 30% 끄고 학습해라
#     if step%3==0:
#         print("cost:{}".format(cost_val))

for step in range(num_of_epoch):   
    _, cost_val = sess.run([train,cost],feed_dict = {X:x_data,
                                                     Y:y_data,
                                                    keep_rate:0.7})
    if step%500==0:
        print("cost:{}".format(cost_val))

cost:1.7707507610321045
cost:0.37824931740760803
cost:0.34818485379219055
cost:0.3582134246826172
cost:0.3402332067489624
cost:0.3368334472179413
cost:0.3369526267051697
cost:0.3380812704563141
cost:0.3254116475582123
cost:0.3379199206829071


In [144]:
predict = tf.cast(H > 0.5,dtype=tf.float32)
# H가 예측값인데, 0.2가 나오면 1에 가까울 확률이 20%인 것, 0.8이면 1에 가까울 확률이 80%
# 몇을 넘으면 1로 볼것인지는 마음대로 잡아도 되지만, 일반적으로 0.5 많이 한다.
# H>0.5 하면 결과 True, False로 떨어지므로 우리가 가지고 있는 데이터 0,1로 표현해 줘야 한다.
# 실제 데이터와 비교하기 위해 값 바꿔주는 과정 
correct = tf.equal(predict, Y)
accuracy = tf.reduce_mean( tf.cast(correct,dtype=tf.float32) )
# correct도 True, False로 떨어짐 -> correct를 실수값으로 다시 바꿔주는 과정

print("정확도:{}".format(sess.run(accuracy,feed_dict={X:test_x_data,
                                                  Y:test_y_data,
                                                  keep_rate:0.7})))


정확도:0.8435754179954529


In [148]:
## 진짜 test
x_data = test_df.values  # 컬럼삭제, 원본유지 

result_H = sess.run(H, feed_dict={X:x_data,
                                 keep_rate:0.7})      

predict = tf.cast(result_H > 0.5,dtype=tf.int32)
result = sess.run(predict)


In [149]:
test_id = pd.read_csv("./data/titanic/test.csv")["PassengerId"]

# concat(): 연결 -> 연결할 것을 list 형태로
s1 = pd.DataFrame(test_id)
s2 = pd.DataFrame(result)
result = pd.concat([s1,s2],axis=1)
#result = result.rename({'Survived':'0'},axis='columns')
result.columns=['PassengerId','Survived']
result

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [150]:
result.to_csv('./data/titanic/submission_deep_dropout.csv',
                 sep=',',
                 na_rep='NaN', 
                 columns = ['PassengerId','Survived'], # columns to write
                 index = False) # do not write index
